In [3]:
import numpy as np
import tensorflow as tf
import keras

from importlib import reload

import sys
sys.path.append('.')

import src.text_generation as text_gen


In [4]:

path_to_file = keras.utils.get_file(
        'shakespeare.txt',
        'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
        )
print(path_to_file)

with open(path_to_file) as f:
    shakespeare_text = f.read()
print(shakespeare_text[:148])

shakespeare_tensor = tf.constant([shakespeare_text])
vocab = list(set(shakespeare_text.lower().strip()))
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, lower=True)
tokenizer.fit_on_texts(shakespeare_text.lower())
n_tokens = len(tokenizer.word_index)

/home/jovyan/.keras/datasets/shakespeare.txt
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



I0000 00:00:1748891127.105865    1467 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1100 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070, pci bus id: 0000:01:00.0, compute capability: 8.9


In [5]:
from keras.models import load_model

# Load the model from the saved path
model = load_model("models/shakespeare_gru_1.keras")
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, None, 128)      │        64,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 39)       │         5,031 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 209,783 (819.47 KB)

 Trainable params: 69,927 (273.15 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 139,856 (546.32 KB)

In [6]:
text = "to be "
sequence = np.array(tokenizer.texts_to_sequences([text])[0])
print(sequence)
print(tokenizer.sequences_to_texts([sequence]))
sequence = sequence - 1
print(sequence)

[ 3  4  1 22  2  1]
['t o   b e  ']
[ 2  3  0 21  1  0]


In [7]:
inp = tf.one_hot(sequence, depth=n_tokens)
print(inp.shape)
inp_tf = tf.reshape(inp, (1, 6, n_tokens))
print(inp_tf.shape)

(6, 39)
(1, 6, 39)


In [8]:
y = model.predict(inp_tf)
print(y.shape)
prediction = y[0, -1, :]
print(prediction.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
(1, 6, 39)
(39,)


I0000 00:00:1748891127.942472    5769 cuda_dnn.cc:529] Loaded cuDNN version 90501


In [9]:
char_id = np.argmax(prediction) + 1
print(char_id)

sequence = np.append(sequence + 1, char_id)
print(sequence)
tokenizer.sequences_to_texts([sequence])

8
[ 3  4  1 22  2  1  8]


['t o   b e   s']

In [10]:
reload(text_gen)
one_hot_fn = lambda x: tf.one_hot(x, depth=n_tokens)
sequence = np.array(tokenizer.texts_to_sequences([text])[0])
sequence = sequence - 1
text_gen.next_token(sequence, model, transform=one_hot_fn, argmax=False, temperature=10)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
prediction: (39,)


np.int64(27)

In [11]:
from src.text_generation import next_token

next_token(sequence, model, transform=one_hot_fn, argmax=False, temperature=10)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
prediction: (39,)


np.int64(4)

In [12]:
reload(text_gen)

text_gen.generate("to be ", model, tokenizer, n_chars=50, temperature=0.3, argmax=True)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
prediction

't o   b e   s o n   o f   t h e   w o r l d   w i t h   t h e   w o r l d \n w i t h   s o m e   s o n   i t   i'